In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("100_Unique_QA_Dataset.csv")

In [4]:
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [5]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace("?","")
  text = text.replace("'","")
  return text.split()

In [6]:
# vocab
vocab = {'<UNK>':0}

In [7]:
def build_vocab(row):
  tokenized_ques = tokenize(row['question'])
  tokenized_ans = tokenize(row['answer'])

  merged_tokens = tokenized_ques + tokenized_ans

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)

In [8]:
df.apply(build_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [9]:
len(vocab)

324

In [10]:
# convert text into indices
def text_to_indices(text,vocab):
  indexed_text = []

  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])
  return indexed_text


In [11]:
text_to_indices("what is paris",vocab)

[1, 2, 7]

In [12]:
import torch
from torch.utils.data import Dataset,DataLoader

In [13]:
class QADataset(Dataset):

  def __init__(self,df,vocab):

    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self,idx):

    numerical_questions = text_to_indices(self.df.iloc[idx]['question'],self.vocab)
    numerical_answers = text_to_indices(self.df.iloc[idx]['answer'],self.vocab)

    return torch.tensor(numerical_questions),torch.tensor(numerical_answers)

In [14]:
dataset = QADataset(df,vocab)

In [15]:
dataset_loader = DataLoader(dataset,batch_size=1,shuffle=True)

In [16]:
import torch.nn as nn

In [17]:
lr = 0.001
epochs = 10

In [18]:
class SimpleRNN(nn.Module):

  def __init__(self,vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim=50)
    self.rnn = nn.RNN(50,64,batch_first=True)
    self.fc = nn.Linear(64,324)

  def forward(self,question):

    embeddings = self.embedding(question)
    hidden,final = self.rnn(embeddings)
    output = self.fc(final.squeeze(0))

    return output

In [19]:
model = SimpleRNN(len(vocab))

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

In [21]:
for i in range(epochs):
  total_loss = 0
  for question,answer in dataset_loader:

    optimizer.zero_grad()
    output = model(question)

    loss = criterion(output,answer[0])

    loss.backward()

    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch :{i + 1},Loss :{total_loss:.4f}")


Epoch :1,Loss :521.0569
Epoch :2,Loss :452.3885
Epoch :3,Loss :372.6422
Epoch :4,Loss :313.6669
Epoch :5,Loss :262.9109
Epoch :6,Loss :216.1302
Epoch :7,Loss :172.8668
Epoch :8,Loss :135.6438
Epoch :9,Loss :104.0765
Epoch :10,Loss :80.6593


In [22]:
dataset[0][0]

tensor([1, 2, 3, 4, 5, 6])

In [23]:
x = nn.Embedding(324,50)

In [24]:
a = x(dataset[0][0])

In [25]:
y = nn.RNN(50,64)

In [26]:
y(a)[1].shape

torch.Size([1, 64])

In [30]:
def predict(model,question,threshold=0.5):
  # convert questions to numerical
  numerical_question = text_to_indices(question,vocab)

  # convert into tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # prediction
  output = model(question_tensor)

  # convert into probabilities
  probs = torch.nn.functional.softmax(output,dim=1)

  # find index of max prob
  value,index = torch.max(probs,dim=1)
  print(value,index)

  if value < threshold:
    print("I dont know")

  print(list(vocab.keys())[index])

In [31]:
predict(model,"What is the capital of France?")

tensor([0.5846], grad_fn=<MaxBackward0>) tensor([7])
paris


In [32]:
list(vocab.keys())[7]

'paris'